In [86]:
import censusdata
import pandas as pd
import numpy as np

In [87]:
#function to make list of all county ids in state (given by census state id)
def county_list(state_number):
    counties = censusdata.geographies(censusdata.censusgeo([('state', state_number), ('county','*')]), 'acs5', 2018)
    county_list = []
    for i in counties.keys():
        county_list.append(counties[i].geo[1][1])
    return county_list

#function to pull defined variables for blocks in specified state, looping over counties (can't pull all blocks in a state)
#(input state id and list of variables)
def block_pull(state_id, variable_list):
    c_list = county_list(state_id)
    for i in range(0, len(c_list)):
        geo = censusdata.censusgeo([('state', state_id), ('county', c_list[i]), ('tract','*'),('block group','*')])
        county_df = censusdata.download('acs5', 2018, geo, variable_list)
        if i == 0:
            data = county_df
        else:
            data = pd.concat([data, county_df], sort=True)
    return data

In [88]:
#just one variable in here for now, need to select and add variables
variables_list = ['B02001_001E','B02001_002E','B02001_003E','B02001_004E','B02001_005E','B02001_006E','B02001_007E',
                  'B02001_008E','B28006_001E','B28006_002E','B28006_008E','B28006_014E','B19013_001E','B17010_002E',
                  'B17010_001E','C16002_001E','C16002_004E','C16002_010E','C16002_013E','B03002_001E','B03002_012E']

block_data = block_pull('39', variables_list)

In [89]:
censusdata.printtable(censusdata.censustable('acs5', 2017, 'B02001'))

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
B02001_001E  | RACE                           | !! Estimate Total                                        | int  
B02001_002E  | RACE                           | !! !! Estimate Total White alone                         | int  
B02001_003E  | RACE                           | !! !! Estimate Total Black or African American alone     | int  
B02001_004E  | RACE                           | !! !! Estimate Total American Indian and Alaska Native a | int  
B02001_005E  | RACE                           | !! !! Estimate Total Asian alone                         | int  
B02001_006E  | RACE                           | !! !! Estimate Total Native Hawaiian and Other Pacific I | int  
B02001_007E  | RACE                           | !! !! Estimate Total Some other race alone   

In [90]:
state_id = [bg.geo[0][1] for bg in bg_index]
county_id = [bg.geo[1][1] for bg in bg_index]
tract_id = [bg.geo[2][1] for bg in bg_index]
bgroup_id= [bg.geo[3][1] for bg in bg_index]
countyname = [bg.name.split(',')[2] for bg in bg_index]
statename = [bg.name.split(',')[3] for bg in bg_index]
schdistr_id = [bg.sumleveldict.get('state> school district (unified)') for bg in bg_index]

In [91]:
bg_index[0].geo

(('state', '39'), ('county', '053'), ('tract', '954000'), ('block group', '3'))

In [92]:
block_data['state_id'] = state_id
block_data['county_id'] = county_id
block_data['tract_id'] = tracts
block_data['bgroup_id'] = bgroup_id
block_data['countyname'] = countyname
block_data['statename'] = statename
block_data['schdistr_id'] = schdistr_id

In [ ]:
cnames = block_data.columns.tolist()

In [66]:
block_data['pk'] = block_data.state + '_'+ block_data.county + '_' +  block_data.tracts + '_'+ block_data.bgroups + 

In [ ]:
block_data = block_data[['pk']+cnames]

In [82]:
block_data.head(2)

,Pop_All,Pop_White,Pop_Black_or_AA,B02001_004E,Pop_Asian,B02001_006E,B02001_007E,B02001_008E,B03002_001E,Pop_Hispanic_Latino,...,C16002_010E,C16002_013E,county,state,tracts,bgroups,ctyname,statename,schdistr,pk
"Block Group 3, Census Tract 9540, Gallia County, Ohio: Summary level: 150, state:39> county:053> tract:954000> block group:3",1348,1145,96,0,82,0,0,25,1348,27,...,0,0,127,39,954000,3,Gallia County,Ohio,970,39_127_954000_3
"Block Group 2, Census Tract 9540, Gallia County, Ohio: Summary level: 150, state:39> county:053> tract:954000> block group:2",621,603,18,0,0,0,0,0,621,0,...,0,0,127,39,954000,2,Gallia County,Ohio,970,39_127_954000_2


In [59]:
block_data.head(1)

,B02001_001E,B02001_002E,B02001_003E,B02001_004E,B02001_005E,B02001_006E,B02001_007E,B02001_008E,B03002_001E,B03002_012E,...,C16002_004E,C16002_010E,C16002_013E,county,state,tracts,bgroups,ctyname,statename,schdistr
"Block Group 3, Census Tract 9540, Gallia County, Ohio: Summary level: 150, state:39> county:053> tract:954000> block group:3",1348,1145,96,0,82,0,0,25,1348,27,...,0,0,0,127,39,954000,3,Gallia County,Ohio,970


In [77]:
var_dict = {  'B02001_001E' : 'Pop_All',
         'B02001_002E' : 'Pop_White',
         'B02001_003E' : 'Pop_Black_or_AA',
         'B02001_005E' : 'Pop_Asian',
         'B03002_012E' : 'Pop_Hispanic_Latino',
         'B28006_001E' : 'Edu_All',
         'B28006_002E' : 'Edu_Less_Than_High_School',
         'B28006_008E' : 'Edu_High_School_Grad',
         'B28006_014E' : 'PCT_Bach_Or_Higher',
         'B19013_001E' : 'Median_Income',
         'B17010_001E' : 'Pov_All',
         'B17010_001E' : 'Pov_Total',
         'B17010_002E' : 'Pov_Poverty'
         }

In [80]:
block_data = block_data.rename(columns=var_dict)
block_data.head(2)

,Pop_All,Pop_White,Pop_Black_or_AA,B02001_004E,Pop_Asian,B02001_006E,B02001_007E,B02001_008E,B03002_001E,Pop_Hispanic_Latino,...,C16002_010E,C16002_013E,county,state,tracts,bgroups,ctyname,statename,schdistr,pk
"Block Group 3, Census Tract 9540, Gallia County, Ohio: Summary level: 150, state:39> county:053> tract:954000> block group:3",1348,1145,96,0,82,0,0,25,1348,27,...,0,0,127,39,954000,3,Gallia County,Ohio,970,39_127_954000_3
"Block Group 2, Census Tract 9540, Gallia County, Ohio: Summary level: 150, state:39> county:053> tract:954000> block group:2",621,603,18,0,0,0,0,0,621,0,...,0,0,127,39,954000,2,Gallia County,Ohio,970,39_127_954000_2


In [37]:
#make more meaningful variables, ie turn raw counts into proportions, etc.
var_dict = {'Pop_All', 'B02001_001E'
            'Pop_White': 'B02001_002E',
            'Pop_Black_or_AA': 'B02001_003E',
            'Pop_Asian': 'B02001_005E',
            'Pop_Hispanic_Latino': 'B03002_012E',
            'Edu_All': 'B28006_001E',
            'Edu_Less_Than_High_School': 'B28006_002E',
            'Edu_High_School_Grad': 'B28006_008E',
            'PCT_Bach_Or_Higher': 'B28006_014E',
            'Median_Income': 'B19013_001E',
            'Pov_All': 'B17010_001E',
            'Pov_Total': 'B17010_001E',
            'Pov_Poverty': 'B17010_002E',
            'PCT_Households_Limited_English': 'C16002_004E'}
            
#make % white column
block_data['PCT_White'] = block_data['B02001_002E']/block_data['B02001_001E']
#make % black and african american
block_data['PCT_Black_or_AA'] = block_data['B02001_003E']/block_data['B02001_002E']
#make % Asian column
block_data['PCT_Asian'] = block_data['B02001_005E']/block_data['B02001_002E']
#make % Hispanic/Latino
block_data['PCT_Hispanic_Latino'] = block_data['B03002_012E']/block_data['B03002_001E']
#make % Less than high school education
block_data['PCT_Less_Than_High_School'] = block_data['B28006_002E']/block_data['B28006_001E']
#make % high school no college
block_data['PCT_High_School_Grad'] = block_data['B28006_008E']/block_data['B28006_001E']
#make % bachelors or higher
block_data['PCT_Bach_Or_Higher'] = block_data['B28006_014E']/block_data['B28006_001E']
#Rename Median Income
block_data['Median_Income'] = block_data['B19013_001E']
#% Households w/ income below poverty line in last 12 months
block_data['PCT_Households_Poverty'] = block_data['B17010_002E']/block_data['B17010_001E']
#% households limited english spoken
block_data['PCT_Households_Limited_English'] = (block_data['C16002_004E']+block_data['C16002_010E']+block_data['C16002_013E'])/block_data['C16002_001E']

#drop old columns
block_data = block_data.drop(variables_list,axis=1)
# get rid of inf values for upload
#block_data.replace([np.inf, -np.inf], np.nan)

,PCT_White,PCT_Black_or_AA,PCT_Asian,PCT_Hispanic_Latino,PCT_Less_Than_High_School,PCT_High_School_Grad,PCT_Bach_Or_Higher,Median_Income,PCT_Households_Poverty,PCT_Households_Limited_English
"Block Group 3, Census Tract 9540, Gallia County, Ohio: Summary level: 150, state:39> county:053> tract:954000> block group:3",0.849407,0.083843,0.071616,0.020030,0.114222,0.599104,0.286674,40563,0.153846,0.0
"Block Group 2, Census Tract 9540, Gallia County, Ohio: Summary level: 150, state:39> county:053> tract:954000> block group:2",0.971014,0.029851,0.000000,0.000000,0.201900,0.776722,0.021378,32045,0.534161,0.0
"Block Group 1, Census Tract 9540, Gallia County, Ohio: Summary level: 150, state:39> county:053> tract:954000> block group:1",0.976000,0.000000,0.024590,0.000000,0.281167,0.527851,0.190981,44167,0.141914,0.0
"Block Group 2, Census Tract 9539, Gallia County, Ohio: Summary level: 150, state:39> county:053> tract:953900> block group:2",0.933537,0.010451,0.000000,0.000000,0.063559,0.731992,0.204449,36759,0.233990,0.0
"Block Group 1, Census Tract 9539, Gallia County, Ohio: Summary level: 150, state:39> county:053> tract:953900> block group:1",0.926049,0.079857,0.000000,0.000000,0.173184,0.702514,0.124302,49100,0.364000,0.0
...,...,...,...,...,...,...,...,...,...,...
"Block Group 1, Census Tract 9659, Perry County, Ohio: Summary level: 150, state:39> county:127> tract:965900> block group:1",1.000000,0.000000,0.000000,0.008512,0.051466,0.736156,0.212378,73052,0.064275,0.0
"Block Group 3, Census Tract 9659, Perry County, Ohio: Summary level: 150, state:39> county:127> tract:965900> block group:3",0.993487,0.001192,0.000000,0.002960,0.046425,0.795729,0.157846,58250,0.078534,0.0
"Block Group 2, Census Tract 9658, Perry County, Ohio: Summary level: 150, state:39> county:127> tract:965800> block group:2",0.967636,0.000000,0.022676,0.000000,0.142174,0.661654,0.196172,76288,0.120141,0.0
"Block Group 3, Census Tract 9661, Perry County, Ohio: Summary level: 150, state:39> county:127> tract:966100> block group:3",1.000000,0.000000,0.000000,0.000000,0.140875,0.690502,0.168623,57768,0.056511,0.0


In [39]:
# SAVE DATA, INDEX IS JUST FOR TESTING
data = block_data
data['index']=range(len(data))
data.head()

,PCT_White,PCT_Black_or_AA,PCT_Asian,PCT_Hispanic_Latino,PCT_Less_Than_High_School,PCT_High_School_Grad,PCT_Bach_Or_Higher,Median_Income,PCT_Households_Poverty,PCT_Households_Limited_English,index
"Block Group 3, Census Tract 9540, Gallia County, Ohio: Summary level: 150, state:39> county:053> tract:954000> block group:3",0.849407,0.083843,0.071616,0.02003,0.114222,0.599104,0.286674,40563,0.153846,0.0,0
"Block Group 2, Census Tract 9540, Gallia County, Ohio: Summary level: 150, state:39> county:053> tract:954000> block group:2",0.971014,0.029851,0.000000,0.00000,0.201900,0.776722,0.021378,32045,0.534161,0.0,1
"Block Group 1, Census Tract 9540, Gallia County, Ohio: Summary level: 150, state:39> county:053> tract:954000> block group:1",0.976000,0.000000,0.024590,0.00000,0.281167,0.527851,0.190981,44167,0.141914,0.0,2
"Block Group 2, Census Tract 9539, Gallia County, Ohio: Summary level: 150, state:39> county:053> tract:953900> block group:2",0.933537,0.010451,0.000000,0.00000,0.063559,0.731992,0.204449,36759,0.233990,0.0,3
"Block Group 1, Census Tract 9539, Gallia County, Ohio: Summary level: 150, state:39> county:053> tract:953900> block group:1",0.926049,0.079857,0.000000,0.00000,0.173184,0.702514,0.124302,49100,0.364000,0.0,4


In [84]:
block_data.to_csv('testlook.csv', index=False)